# Import Libraries

In [1]:
from google.colab import drive
import os
import numpy as np
import pandas as pd
import cv2
import math
from git import Repo

# Mount Drive

In [2]:
drive.mount('/content/drive/')

Mounted at /content/drive/


# Clone Repository

In [3]:
%cd "/content/drive/My Drive"
if not os.path.exists('yogapose-classification-ocr'):
  !git clone https://github.com/AkienMain/yogapose-classification-ocr.git

/content/drive/My Drive
Cloning into 'yogapose-classification-ocr'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 11 (delta 2), reused 11 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (11/11), 1.55 MiB | 5.73 MiB/s, done.
Resolving deltas: 100% (2/2), done.


# Download Dataset

https://github.com/Manoj-2702/Yoga_Poses-Dataset

The Yoga-Pose Dataset is a comprehensive collection of yoga pose images, pose landmarks, and calculated angles for various yoga poses.

In [4]:
%cd "/content/drive/My Drive/yogapose-classification-ocr"
original_dataset_folder = 'Yoga_Poses-Dataset'
if not os.path.exists(original_dataset_folder):
  Repo.clone_from('https://github.com/Manoj-2702/Yoga_Poses-Dataset.git', original_dataset_folder)

/content/drive/My Drive/yogapose-classification-ocr


# Extract position data

In [5]:
# Posename dictionary
posename_dic1 = {
  0: 'ArdhaChandrasana',
  1: 'BaddhaKonasana',
  2: 'DownwardDog',
  3: 'Natarajasana',
  4: 'Triangle',
  5: 'UtkataKonasana',
  6: 'Veerabhadrasana',
  7: 'Vrukshana'
}

# Dataframe for position data
df_pos = pd.DataFrame([])
for k in posename_dic1.keys():
  posename = posename_dic1[k]
  file = f'Results/Combined Dataset/{posename}_Combined.csv'
  df_i = pd.read_csv(f'{original_dataset_folder}/{file}', index_col=0)
  df_i['pose_num'] = k
  df_pos = pd.concat([df_pos, df_i], ignore_index=True)

# Drop columns we don't use
  # angle
  # z (estimated depth)
  # vis (estimated visuality)
  # less important joint points
    # EYE, MOUTH, EAR, PINKY, THUMB, HEEL
drop_condition = \
  df_pos.columns.str.contains('angle') + \
  df_pos.columns.str.contains('_z') + \
  df_pos.columns.str.contains('_vis') + \
  df_pos.columns.str.contains('EYE') + \
  df_pos.columns.str.contains('MOUTH') + \
  df_pos.columns.str.contains('EAR') + \
  df_pos.columns.str.contains('PINKY') + \
  df_pos.columns.str.contains('THUMB') + \
  df_pos.columns.str.contains('HEEL')
df_pos = df_pos.drop(df_pos.columns[drop_condition], axis=1)

# Create position information
  # Center hip
  # Center shoulder
df_pos['CENTER_HIP_x'] = (df_pos['LEFT_HIP_x'] + df_pos['RIGHT_HIP_x'])/2
df_pos['CENTER_HIP_y'] = (df_pos['LEFT_HIP_y'] + df_pos['RIGHT_HIP_y'])/2
df_pos['CENTER_SHOULDER_x'] = (df_pos['LEFT_SHOULDER_x'] + df_pos['RIGHT_SHOULDER_x'])/2
df_pos['CENTER_SHOULDER_y'] = (df_pos['LEFT_SHOULDER_y'] + df_pos['RIGHT_SHOULDER_y'])/2

# Reorder column
new_col = df_pos.columns
new_col = new_col.drop('pose_num').append(pd.Index(['pose_num']))
df_pos = df_pos.reindex(columns=new_col)

# Set image file name column
df_pos['image_file'] = df_pos['Label'].str.replace(r'.*\/', r'', regex=True)
df_pos = df_pos.drop(columns=['Label'], axis=1)

# Show dataframe
pd.options.display.max_columns = 200
df_pos.head()



,NOSE_x,NOSE_y,LEFT_SHOULDER_x,LEFT_SHOULDER_y,RIGHT_SHOULDER_x,RIGHT_SHOULDER_y,LEFT_ELBOW_x,LEFT_ELBOW_y,RIGHT_ELBOW_x,RIGHT_ELBOW_y,LEFT_WRIST_x,LEFT_WRIST_y,RIGHT_WRIST_x,RIGHT_WRIST_y,LEFT_INDEX_x,LEFT_INDEX_y,RIGHT_INDEX_x,RIGHT_INDEX_y,LEFT_HIP_x,LEFT_HIP_y,RIGHT_HIP_x,RIGHT_HIP_y,LEFT_KNEE_x,LEFT_KNEE_y,RIGHT_KNEE_x,RIGHT_KNEE_y,LEFT_ANKLE_x,LEFT_ANKLE_y,RIGHT_ANKLE_x,RIGHT_ANKLE_y,LEFT_FOOT_INDEX_x,LEFT_FOOT_INDEX_y,RIGHT_FOOT_INDEX_x,RIGHT_FOOT_INDEX_y,CENTER_HIP_x,CENTER_HIP_y,CENTER_SHOULDER_x,CENTER_SHOULDER_y,pose_num,image_file
0,0.353510,0.456924,0.399010,0.436970,0.397380,0.577131,0.401657,0.337513,0.395152,0.687620,0.395510,0.238502,0.384274,0.788146,0.391797,0.205824,0.376810,0.817950,0.536427,0.460180,0.526439,0.526197,0.648127,0.471818,0.529578,0.678412,0.751680,0.481057,0.519321,0.844806,0.779012,0.474065,0.478309,0.877947,0.531433,0.493189,0.398195,0.507050,0,1.jpg
1,0.679009,0.484446,0.622567,0.597629,0.635454,0.442142,0.620872,0.698814,0.626558,0.335369,0.620745,0.806753,0.635187,0.224608,0.628703,0.833764,0.644565,0.190127,0.517898,0.494339,0.516665,0.407607,0.507310,0.639865,0.412838,0.400096,0.512782,0.808377,0.325540,0.388523,0.551207,0.835450,0.294297,0.384578,0.517282,0.450973,0.629011,0.519885,0,2.jpg
2,0.290658,0.455510,0.351107,0.411933,0.355504,0.577494,0.360214,0.279359,0.359991,0.717141,0.362361,0.162034,0.358026,0.839014,0.361021,0.122845,0.352811,0.877158,0.512775,0.384104,0.508220,0.473788,0.659123,0.388002,0.518612,0.663860,0.788982,0.383995,0.504995,0.858937,0.848468,0.384954,0.455744,0.904862,0.510497,0.428946,0.353305,0.494713,0,3.jpg
3,0.201762,0.541465,0.265578,0.486993,0.274992,0.621992,0.287076,0.383804,0.282587,0.727878,0.280733,0.278816,0.257223,0.834759,0.272934,0.244108,0.237709,0.862821,0.463075,0.464473,0.454494,0.533345,0.622319,0.418257,0.475272,0.688443,0.767145,0.384323,0.470569,0.841834,0.846569,0.399111,0.408666,0.883773,0.458785,0.498909,0.270285,0.554492,0,4.png
4,0.267801,0.449926,0.331943,0.403497,0.338687,0.582700,0.338409,0.277516,0.345836,0.718116,0.322631,0.168933,0.341114,0.845069,0.316322,0.132234,0.333688,0.881144,0.491716,0.380604,0.490575,0.480239,0.613886,0.317145,0.506175,0.669797,0.725107,0.252064,0.503394,0.867474,0.770912,0.218055,0.449379,0.915585,0.491146,0.430421,0.335315,0.493098,0,5.jpeg


# Extract image data

In [6]:
# Posename dictionary
posename_dic2 = {
  0: 'ArdhaChandrasana',
  1: 'BaddhaKonasana',
  2: 'Downward_dog',
  3: 'Natarajasana',
  4: 'Triangle',
  5: 'UtkataKonasana',
  6: 'Veerabhadrasana',
  7: 'Vrukshasana'
}

# Dataframe for image data
df_im = pd.DataFrame(columns=['width', 'height'])
for k in posename_dic2.keys():
  im_path = f'Yoga_Poses-Dataset/TRAIN/{posename_dic2[k]}/Images'
  im_names = os.listdir(im_path)
  image_data = np.empty([len(im_names), 2]).astype(int)

  # Read image and get width and height
  for j in range(len(im_names)):
    img = cv2.imread(f'{im_path}/{im_names[j]}', cv2.IMREAD_GRAYSCALE)
    image_data[j] = np.asarray((img.shape[1], img.shape[0]))

  df_i = pd.DataFrame(image_data, columns=['width', 'height'])
  df_i['pose_num'] = k
  df_i['image_file'] = pd.Series(im_names)
  df_im = pd.concat([df_im, df_i])

# Set pose_num as int
df_im = df_im.astype({'pose_num': int})

# Reset index
df_im = df_im.reset_index(drop=True)


/tmp/ipython-input-529621726.py:18: RuntimeWarning: invalid value encountered in cast
  image_data = np.empty([len(im_names), 2]).astype(int)


# Merge position and image data

In [7]:
# Merge two dataframes
# Use two columns ('pose_num', 'image_file') as keys
df = pd.merge(df_pos, df_im, how='outer', on=['pose_num', 'image_file'])

# Drop image file column
df = df.drop(columns=['image_file'], axis=1)

# Change unit from relative value (0 to 1) to absolute value (pixel)
for col in df.columns:
  if col.__contains__('_x'):
    df[col] = df[col] * df['width']
  if col.__contains__('_y'):
    df[col] = df[col] * df['height']

# Drop NaN row
df = df.dropna()
df = df.reset_index(drop=True)

# Create angle data

In [8]:
# Create angle info myself instead original angle info

# Definition of segments
TRUNK = ['CENTER_SHOULDER','CENTER_HIP']
UPPER_ARM_L = ['LEFT_SHOULDER','LEFT_ELBOW']
UPPER_ARM_R = ['RIGHT_SHOULDER','RIGHT_ELBOW']
LOWER_ARM_L = ['LEFT_ELBOW','LEFT_WRIST']
LOWER_ARM_R = ['RIGHT_ELBOW','RIGHT_WRIST']
THIGH_L = ['LEFT_HIP','LEFT_KNEE']
THIGH_R = ['RIGHT_HIP','RIGHT_KNEE']
SHANK_L = ['LEFT_KNEE','LEFT_ANKLE']
SHANK_R = ['RIGHT_KNEE','RIGHT_ANKLE']

# Definition of angles
angle_def = {
  'Angle_upper_arms' : [UPPER_ARM_L, UPPER_ARM_R],
  'Angle_elbow_L' : [UPPER_ARM_L, LOWER_ARM_L],
  'Angle_elbow_R' : [UPPER_ARM_R, LOWER_ARM_R],
  'Angle_thighs' : [THIGH_L, THIGH_R],
  'Angle_knee_L' : [THIGH_L, SHANK_L],
  'Angle_knee_R' : [THIGH_R, SHANK_R],
  'Angle_shoulder_L' : [TRUNK, UPPER_ARM_L],
  'Angle_shoulder_R' : [TRUNK, UPPER_ARM_R],
  'Angle_hip_L' : [TRUNK, THIGH_L],
  'Angle_hip_R' : [TRUNK, THIGH_R]
}

# Create angle info using dot product of two vectors and arccos
for i in df.index:
  for k, v in angle_def.items():

    # Vector
    v1 = [df.loc[i,f'{v[0][1]}_x']-df.loc[i,f'{v[0][0]}_x'], df.loc[i,f'{v[0][1]}_y']-df.loc[i,f'{v[0][0]}_y']]
    v2 = [df.loc[i,f'{v[1][1]}_x']-df.loc[i,f'{v[1][0]}_x'], df.loc[i,f'{v[1][1]}_y']-df.loc[i,f'{v[1][0]}_y']]

    # Unit vector
    uv1 = v1 / np.linalg.norm(v1)
    uv2 = v2 / np.linalg.norm(v2)

    # Angle in degree
    dot_product = np.dot(uv1, uv2)
    angle = math.degrees(np.arccos(dot_product))

    df.loc[i,k] = angle

# Export dataset

In [9]:
# Reorder columns
new_col = df.columns
new_col = new_col.drop('width').append(pd.Index(['width']))
new_col = new_col.drop('height').append(pd.Index(['height']))
new_col = new_col.drop('pose_num').append(pd.Index(['pose_num']))
df = df.reindex(columns=new_col)

# change data type
for col in df.columns:
  if col.__contains__('_x') or col.__contains__('_y'):
    df = df.astype({col: float})
  if col.__contains__('width') or col.__contains__('height'):
    df = df.astype({col: int})

# Export
df.to_csv('dataset.csv', float_format='%.2f')

# show
df.head()

,NOSE_x,NOSE_y,LEFT_SHOULDER_x,LEFT_SHOULDER_y,RIGHT_SHOULDER_x,RIGHT_SHOULDER_y,LEFT_ELBOW_x,LEFT_ELBOW_y,RIGHT_ELBOW_x,RIGHT_ELBOW_y,LEFT_WRIST_x,LEFT_WRIST_y,RIGHT_WRIST_x,RIGHT_WRIST_y,LEFT_INDEX_x,LEFT_INDEX_y,RIGHT_INDEX_x,RIGHT_INDEX_y,LEFT_HIP_x,LEFT_HIP_y,RIGHT_HIP_x,RIGHT_HIP_y,LEFT_KNEE_x,LEFT_KNEE_y,RIGHT_KNEE_x,RIGHT_KNEE_y,LEFT_ANKLE_x,LEFT_ANKLE_y,RIGHT_ANKLE_x,RIGHT_ANKLE_y,LEFT_FOOT_INDEX_x,LEFT_FOOT_INDEX_y,RIGHT_FOOT_INDEX_x,RIGHT_FOOT_INDEX_y,CENTER_HIP_x,CENTER_HIP_y,CENTER_SHOULDER_x,CENTER_SHOULDER_y,Angle_upper_arms,Angle_elbow_L,Angle_elbow_R,Angle_thighs,Angle_knee_L,Angle_knee_R,Angle_shoulder_L,Angle_shoulder_R,Angle_hip_L,Angle_hip_R,width,height,pose_num
0,1589.382133,1370.772063,1793.951114,1310.910435,1786.620723,1731.392205,1805.848199,1012.539624,1776.601396,2062.859595,1778.215186,715.506525,1727.696466,2364.438057,1761.521083,617.471874,1694.139833,2453.848959,2411.774277,1380.540402,2366.868184,1578.592479,2913.980417,1415.452659,2380.981213,2035.237311,3379.552012,1443.171501,2334.867594,2534.418942,3502.439400,1422.195465,2150.479431,2633.841276,2389.321230,1479.566440,1790.285918,1521.151320,179.447990,7.598307,7.479722,84.253107,0.569468,7.048131,83.745539,95.702451,7.947765,92.200872,4496,3000,0
1,1629.621077,654.002327,1494.161366,806.799244,1525.089742,596.891553,1490.091991,943.399269,1503.740359,452.748741,1489.787436,1089.116200,1524.448728,303.221289,1508.887054,1125.582066,1546.957111,256.671088,1242.955970,667.358078,1239.996242,550.269596,1217.543220,863.817322,990.810012,540.129519,1230.675602,1091.308344,781.296014,524.506550,1322.895670,1127.857330,706.312536,519.180768,1241.476106,608.813837,1509.625554,701.845399,169.868656,1.586611,16.309839,84.959738,10.674349,1.934282,107.427304,62.441352,101.763166,16.803428,2400,1350,0
2,581.315040,607.650288,702.213944,549.518629,711.007476,770.376951,720.428348,372.664486,719.981730,956.665978,724.722028,216.153959,716.052890,1119.245144,722.041130,163.875630,705.621720,1170.128753,1025.549174,512.394277,1016.440272,632.033700,1318.245410,517.594799,1037.223578,885.588953,1577.964068,512.249007,1009.989620,1145.821435,1696.936608,513.528763,911.488234,1207.086526,1020.994723,572.213989,706.610710,659.947790,171.361752,4.308776,4.142348,84.296177,2.197057,10.660314,68.527181,102.834571,16.610500,100.906676,2000,1334,0
3,183.603710,492.732811,241.675738,443.163725,250.242331,566.012496,261.238885,349.261607,257.153864,662.368740,255.467427,253.722519,234.073373,759.630687,248.369904,222.137974,216.315476,785.166986,421.398695,422.670656,413.589106,485.344207,566.310437,380.614031,432.497845,626.482863,698.102256,349.734039,428.218183,766.068785,770.377574,363.190606,371.885938,804.233619,417.493900,454.007431,245.959034,504.588110,164.128850,15.225412,17.452240,98.553271,2.996890,9.386759,61.802324,102.326526,0.245357,98.798628,910,910,0
4,75.787713,80.086752,93.939930,71.822404,95.848388,103.720613,95.769732,49.397902,97.871499,127.824656,91.304607,30.070160,96.535224,150.422333,89.519067,23.537606,94.433769,156.843587,139.155703,67.747480,138.832797,85.482559,173.729839,56.451819,143.247486,119.223813,205.205201,44.867358,142.460437,154.410397,218.168144,38.813847,127.174340,162.974148,138.994250,76.615019,94.894159,87.771509,170.537364,17.673247,8.181878,100.638443,2.113413,8.735579,71.138207,99.399157,3.895764,96.742680,283,178,0
